In [ ]:
# access to google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# create a directory in drive
import os
print(os.getcwd())

In [ ]:
%cd "/content/gdrive/MyDrive//2023ECGChallenge"

### *load dataset*

In [ ]:
import numpy as np
import pandas as pd

x_train = pd.read_csv('x_train_lead_3.csv', index_col= 0)
y_train = pd.read_csv('y_train_lead_3.csv', index_col= 0)

# print(x_train.shape)
# print(y_train.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_train.iloc[0,:])

In [ ]:
# mark accute/not accute
y_train = np.array(y_train['not_accute'])
y_train = y_train.reshape(-1, 1)

### *model creation*

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPool1D, ReLU, Input, Flatten, Dense, Add, BatchNormalization, AveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation

# define residual block
def residual_block(x):
    # define the two Conv1D layers in the residual block
    y = Conv1D(filters=32, kernel_size=5, padding='same')(x)
    y = BatchNormalization()(y)
    y = ReLU()(y)
    
    y = Conv1D(filters=32, kernel_size=5, padding='same')(y)
    y = BatchNormalization()(y)
    
    # add residual skip connection
    y = Add()([y, x])
    
    # add ReLU function, maxpooling
    y = MaxPool1D(5, 2)(y)
    
    return y

# define input layer shape
inputs = Input(shape=(1000, 1))

x = Conv1D(filters=32, kernel_size=5, strides=1)(inputs)

x = residual_block(x)
x = residual_block(x)
x = residual_block(x)
x = residual_block(x)
x = residual_block(x)

x = Flatten()(x)

x = Dense(32, activation='gelu')(x)
x = Dense(32, activation='gelu')(x)

outputs = Dense(1, activation='sigmoid')(x)

# model definition
model = Model(inputs, outputs)

In [ ]:
# get model information
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
import math

def lr_schedule(epoch):
    # define simple learning rate decay
    lr = 0.001 * math.pow(0.1, math.floor(epoch / 10))
    return lr

# print('Decaying Rate')
# print(lr_schedule(9), lr_schedule(10), lr_schedule(19), lr_schedule(20))

lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

# compile model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

filepath = "best_model_accutebinary.h5"
# define early_stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=30)
# define checkpoint
model_checkpoint = ModelCheckpoint(filepath, save_best_only=True, save_weights_only=False, monitor='val_loss', mode='min')

In [ ]:
# check gpu usage
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

with tf.device(device_name):
    history = model.fit(x=x_train,
                          y=y_train,           
                          validation_split= 0.1,
                          batch_size=128,         
                          epochs=100,            
                          verbose=2,
                        callbacks=[lr_scheduler, early_stopping, model_checkpoint])